In [1]:
from __future__ import print_function
import os
import sys
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], "src"))

## Prepare single robot scene

In [2]:
from pkg.utils.test_scripts import *
from pkg.controller.robot_config import *

ROBOT_TYPE = RobotType.panda
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_buillder, pscene = prepare_single_robot_scene(
    ROBOT_TYPE=ROBOT_TYPE, ROBOT_NAME=ROBOT_NAME, TOOL_LINK=TOOL_LINK, TOOL_XYZ=TOOL_XYZ, TOOL_RPY=TOOL_RPY)
gscene = pscene.gscene

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


## Prepare dummy object

In [3]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool
grip = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## Prepare planner

In [4]:
from pkg.planning.constraint.constraint_common import BindingTransform
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene=pscene)
checker = MoveitPlanner(pscene=pscene)

In [5]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck=gcheck)

mplan.motion_filters = [gcheck]
checker.motion_filters = [gcheck, rcheck, lcheck]

In [6]:
WorkPlane.RTH_MIN = (0.3, 0, 0.2)
WorkPlane.RTH_MAX = (0.3, 0, 0.2)
WorkPlane.RPY_MIN = (0, 0, 0)
WorkPlane.RPY_MAX = (0, 0, 0)

## set workplane
wp = WorkPlane(gscene, "wp", floor_height=0)
wp.geometry.set_offset_tf((0.6, 0.0, 0.02), np.identity(3))
wp.geometry.color = (0.9,0.9,0.9,1)
wp.geometry.dims = (0.5,1,0.05)
gscene.update_markers_all()
pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

pole = Pole(gscene, "pole")
pole.geometry.set_offset_tf((0.5037082845331903, 0.45729871295844005, 0.0), 
                            np.array([[ 0.77924002, -0.62672561,  0.        ],
                                      [ 0.62672561,  0.77924002,  0.        ],
                                      [ 0.        ,  0.        ,  1.        ]]))
pole.geometry.dims = (0.2, 0.3, 4.)

bar = Bar(gscene, "bar")
bar.geometry.set_offset_tf((0.2818130430044775, -0.30949004364604027, 0.8162341036575937), 
                           np.array([[ 0.67327226,  0.73939466,  0.        ],
                                     [-0.73939466,  0.67327226,  0.        ],
                                     [ 0.        ,  0.        ,  1.        ]]))
bar.geometry.dims = (0.2, 4., 0.3)

gscene.update_markers_all()

In [7]:
HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
crob = pscene.combined_robot
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)
gscene.show_pose(HOME_POSE)

In [8]:
idx = 0
width = float(wp.geometry.dims[1])
depth = float(wp.geometry.dims[0])
correctness = []
for x in np.arange(-depth/2+depth/5, depth/2, depth/5):
    for y in np.arange(-width/2+width/10, width/2, width/10):
        idx += 1
        print("==== {} ====".format(np.round((x,y), 2)))
        obj_name = "cyl_{}".format(idx)
        cyl_dims = (0.05,0.05,0.2)
        clearance = 0.001
        cyl = gscene.create_safe(GEOTYPE.CYLINDER, obj_name, link_name="base_link", dims=cyl_dims, 
                                 center=np.add(wp.geometry.center, (x,y,wp.geometry.dims[2]/2+cyl_dims[2]/2+clearance)), 
                                 rpy=(0,0,0), color=(1,1,0.0,0.2), display=True, fixed=False, collision=True)

        xy_rel = np.abs(np.matmul(SE3_inv(pole.geometry.Toff), cyl.Toff)[:2,3]) - np.divide(pole.geometry.dims[:2], 2)
        r_cyl = cyl.dims[0]/2
        if all(xy_rel<r_cyl) or np.linalg.norm(xy_rel)<r_cyl:
            cyl.color = (1,0,0,0.2)
            cyl.collision = False
            gscene.update_markers_all()
            print("colliding")
            continue
        continue
            
        cyl_obj = pscene.create_subject(obj_name, obj_name, CylinderObject)
            
        initial_state = pscene.initialize_state(pscene.combined_robot.home_pose)
        print(pscene.subject_name_list)
        print(initial_state.node)

        goal = ("grip0",)
        available_binding_dict = pscene.get_available_binding_dict(initial_state, to_node=goal, Q_dict=crob.home_dict)
        to_state = pscene.sample_leaf_state(initial_state, to_node=goal, available_binding_dict=available_binding_dict)

        btf = to_state.binding_state[obj_name]
        obj, handle, actor = btf.get_instance_chain(pscene)

        color_dict = {(True): (0,0,1,0.3), (False): (1,0,0,0.1)}
        redun_len = cyl.dims[2]/2-0.05
        uxw_list = []
        gscene.show_pose(crob.home_pose)
        idx_bd = 0
        for u_ in np.arange(-np.pi, np.pi, np.pi/4):
            for x_ in np.arange(-redun_len, redun_len+1e-3, redun_len):
                for w_ in np.arange(-np.pi/4, np.pi/4+1e-3, np.pi/4):
                    idx_bd += 1
                    btf.redundancy[obj_name+"_side_g"]["u"] = u_
                    btf.redundancy[obj_name+"_side_g"]["x"] = x_
                    btf.redundancy["grip0"]["w"] = w_
                    btf.set_redundancy(btf.redundancy, handle, actor)
                    btf.update_transforms(handle, actor)
                    print("---- {} ----".format(np.round((u_,x_,w_), 2)))

                    feas = checker.plan_transition(initial_state, to_state, test_filters_only=True)
                    traj, Q, err, succ, chains = mplan.plan_transition(initial_state, to_state, timeout = 0.5)
                    correctness.append(feas==succ)
                    pscene.show_binding(btf, axis="y", color=color_dict[succ], dims=(0.05,0.01,0.01), idx=str(idx_bd))
                    print(succ)
            print("====")
            print("{} %".format(np.round(np.mean(correctness)*100, 2)))
        print("====")
        print("{} %".format(np.round(np.mean(correctness)*100, 2)))
        pscene.clear_subjects()
        cyl.collision = False
print("=================")
print("{} %".format(np.round(np.mean(correctness)*100, 2)))

==== [-0.15 -0.4 ] ====
==== [-0.15 -0.3 ] ====
==== [-0.15 -0.2 ] ====
==== [-0.15 -0.1 ] ====
==== [-0.15 -0.  ] ====
==== [-0.15  0.1 ] ====
==== [-0.15  0.2 ] ====
==== [-0.15  0.3 ] ====
==== [-0.15  0.4 ] ====
colliding
==== [-0.05 -0.4 ] ====
==== [-0.05 -0.3 ] ====
==== [-0.05 -0.2 ] ====
==== [-0.05 -0.1 ] ====
==== [-0.05 -0.  ] ====
==== [-0.05  0.1 ] ====
==== [-0.05  0.2 ] ====
==== [-0.05  0.3 ] ====
colliding
==== [-0.05  0.4 ] ====
colliding
==== [ 0.05 -0.4 ] ====
==== [ 0.05 -0.3 ] ====
==== [ 0.05 -0.2 ] ====
==== [ 0.05 -0.1 ] ====
==== [ 0.05 -0.  ] ====
==== [0.05 0.1 ] ====
==== [0.05 0.2 ] ====
==== [0.05 0.3 ] ====
==== [0.05 0.4 ] ====
colliding
==== [ 0.15 -0.4 ] ====
==== [ 0.15 -0.3 ] ====
==== [ 0.15 -0.2 ] ====
==== [ 0.15 -0.1 ] ====
==== [ 0.15 -0.  ] ====
==== [0.15 0.1 ] ====
==== [0.15 0.2 ] ====
==== [0.15 0.3 ] ====
==== [0.15 0.4 ] ====
nan %


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
bar.geometry.color = (0,0,0,0)
gscene.update_markers_all()

In [9]:
gscene.show_pose([0, -np.pi / 8, 0, -np.pi / 2, 0, np.pi / 2, np.pi / 2])

In [51]:
gscene.show_motion(traj)
time.sleep(0.1)
gscene.show_pose(Q)

In [17]:
gscene.clear_highlight()